<a href="https://colab.research.google.com/github/rickiepark/MLQandAI/blob/main/supplementary/q11-conv-size/q11-conv-size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11장의 예제 코드

In [1]:
import torch
print(f"파이토치 버전: {torch.__version__}")

파이토치 버전: 2.4.0+cu121


## 1) 합성곱 신경망

In [2]:
class PyTorchCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 5, kernel_size=5, stride=1), # 5 * (5*5 * 3) + 5
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=2),
            torch.nn.Conv2d(5, 12, kernel_size=3, stride=1),  # 12 * (3*3 * 5) + 12
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=3, stride=2),
            torch.nn.ReLU(),
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(192, 128), # 192 * 128 + 128
            torch.nn.ReLU(),
            torch.nn.Linear(128, num_classes), # 128 * 10 + 10
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

## 2) 파라미터 개수 계산하기

### 2 a) 직접

In [3]:
# 합성곱 층 파라미터
conv_part =  (5 * (5*5 * 3) + 5 ) + ( 12 * (3*3 * 5) + 12 )
print(conv_part)

932


In [4]:
# 완전 연결 층 파라미터
fc_part = 192*128+128 + 128*10+10
print(fc_part)

25994


In [5]:
# 총 파라미터
print(conv_part + fc_part)

26926


### 2 b) .parameters() 사용하기

In [6]:
model = PyTorchCNN(10)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
# 합성곱 층 파라미터
count_parameters(model.features)

932

In [8]:
# 완전 연결 층 파라미터
count_parameters(model.classifier)

25994

In [9]:
# 총 파라미터
count_parameters(model)

26926

### 2 c) 메모리 사용량 계산하기

In [10]:
import sys

def calculate_size(model):
    return sum(p.element_size()*p.numel() for p in model.parameters())

size_in_bytes = calculate_size(model)
size_in_megabytes = size_in_bytes * 1e-6

print(f"{size_in_megabytes: .2f} Mb")

 0.11 Mb


### 2 d) torchinfo 라이브러리 사용하기

In [12]:
# using https://github.com/TylerYep/torchinfo
!pip install torchinfo

import torchinfo

print(f"Torchinfo 버전: {torchinfo.__version__}")

batch_size = 16
torchinfo.summary(model, input_size=(batch_size, 3, 32, 32))

Torchinfo 버전: 1.8.0


Layer (type:depth-idx)                   Output Shape              Param #
PyTorchCNN                               [16, 10]                  --
├─Sequential: 1-1                        [16, 12, 4, 4]            --
│    └─Conv2d: 2-1                       [16, 5, 28, 28]           380
│    └─ReLU: 2-2                         [16, 5, 28, 28]           --
│    └─MaxPool2d: 2-3                    [16, 5, 12, 12]           --
│    └─Conv2d: 2-4                       [16, 12, 10, 10]          552
│    └─ReLU: 2-5                         [16, 12, 10, 10]          --
│    └─AvgPool2d: 2-6                    [16, 12, 4, 4]            --
│    └─ReLU: 2-7                         [16, 12, 4, 4]            --
├─Sequential: 1-2                        [16, 10]                  --
│    └─Flatten: 2-8                      [16, 192]                 --
│    └─Linear: 2-9                       [16, 128]                 24,704
│    └─ReLU: 2-10                        [16, 128]                 --
│    └─Li

## 3) ADAM 옵티마이저

In [13]:
optimizer = torch.optim.Adam(model.parameters())

In [14]:
optimizer.param_groups.count

<function list.count(value, /)>

In [15]:
sum(p.numel() for p in optimizer.param_groups[0]['params'])

26926

## 4) 배치 정규화

In [16]:
class PyTorchCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 5, kernel_size=5, stride=1),
            torch.nn.BatchNorm2d(5),  # NEW!
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=5, stride=2),
            torch.nn.Conv2d(5, 12, kernel_size=3, stride=1),
            torch.nn.BatchNorm2d(12),  # NEW!
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=3, stride=2),
            torch.nn.ReLU(),
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(192, 128),
            torch.nn.BatchNorm1d(128),  # NEW!
            torch.nn.ReLU(),
            torch.nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [17]:
model = PyTorchCNN(10)

torchinfo.summary(model, input_size=(batch_size, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
PyTorchCNN                               [16, 10]                  --
├─Sequential: 1-1                        [16, 12, 4, 4]            --
│    └─Conv2d: 2-1                       [16, 5, 28, 28]           380
│    └─BatchNorm2d: 2-2                  [16, 5, 28, 28]           10
│    └─ReLU: 2-3                         [16, 5, 28, 28]           --
│    └─MaxPool2d: 2-4                    [16, 5, 12, 12]           --
│    └─Conv2d: 2-5                       [16, 12, 10, 10]          552
│    └─BatchNorm2d: 2-6                  [16, 12, 10, 10]          24
│    └─ReLU: 2-7                         [16, 12, 10, 10]          --
│    └─AvgPool2d: 2-8                    [16, 12, 4, 4]            --
│    └─ReLU: 2-9                         [16, 12, 4, 4]            --
├─Sequential: 1-2                        [16, 10]                  --
│    └─Flatten: 2-10                     [16, 192]                 --
│    └─Linear